In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

from urllib.request import urlopen # request package
from bs4 import BeautifulSoup
import pandas as pd

ServiceKey='29jTU1S%2B7bDI1Rb2LV0S5D0d3Xn%2Fji9T3k%2FWjsLR%2FyI9O3wEU0atKsr9hj9ATsfIpKgWmV8k8Ll67O%2F73Nyj6w%3D%3D'
month=202312
gu_code=11230

numOfRows=1000
url="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?LAWD_CD="+str(gu_code)+"&DEAL_YMD="+str(month)+"&numOfRows="+str(numOfRows)+"&serviceKey="+str(ServiceKey)

result=urlopen(url) 
house=BeautifulSoup(result,'lxml-xml')

te=house.find_all("item")
print(te[0])

data=[]

for i in range(len(te)):
    deposit=te[i].보증금액.string.strip() 
    m_rent=te[i].월세금액.string.strip() 
    built_yr=te[i].건축년도.string.strip()
    dong_name=te[i].법정동.string.strip() 
    apt_name=te[i].아파트.string.strip()
    size=te[i].전용면적.string.strip() 
    gu_code=te[i].지역코드.string.strip()
    
    total=[month,  deposit, m_rent, built_yr, dong_name, apt_name, size, gu_code]
    data.append(total)
    
df=pd.DataFrame(data, columns=['monoth', 'deposit','m_rent', 'built_yr', 
'dong_name', 'apt_name', 'size', 'gu_code']) 

df.to_csv('c:/temp100/dongdaemungu_202312.csv', encoding='utf-8-sig')

이걸로 불러온 데이터를 다운받아서 그 csv 파일을 다운받아놓은 거임 그걸 이제 불러올거임

In [2]:
# 동대문구 2023년 12월 아파트, 전월세 거래 데이터
# dongdaemungu_202312.csv

df=pd.read_csv("c:/temp/dongdaemungu_202312.csv")
df

,Unnamed: 0,monoth,deposit,m_rent,built_yr,dong_name,apt_name,size,gu_code
0,0,202312,"1,000",80,2018,신설동,포레스트힐시티,14.750,11230
1,1,202312,"1,000",77,2017,신설동,신설동지웰홈스,19.700,11230
2,2,202312,"1,000",70,2017,신설동,신설동지웰홈스,13.750,11230
3,3,202312,"16,200",10,2018,신설동,포레스트힐시티,15.250,11230
4,4,202312,"3,000",56,2018,신설동,포레스트힐시티,15.210,11230
...,...,...,...,...,...,...,...,...,...
670,670,202312,"37,000",0,2002,이문동,중앙하이츠빌,59.400,11230
671,671,202312,"5,000",100,2000,이문동,쌍용,59.990,11230
672,672,202312,"37,000",0,2002,이문동,중앙하이츠빌,59.400,11230
673,673,202312,"54,000",0,2001,이문동,이문삼성래미안아파트,149.934,11230


In [3]:
# 전월세 거래가 총 675건 발생함
# size는 전용면적, 저기에 30%를 가산해야 분양할 때 그 면적
# 전월세는 deposit(보증금) 금액이 엄청 다름 -> 표준화 시켜야함 = 월세가격을 전세로 전환시켜야함 or 전세를 월세로 전환시켜야함 => 전월세 전환
# 아... 안들었음... 전월세 전환 어케하라고

### 전세-월세 전환 공식 
- 전환된 전세금*이자율(0.05)/12=월세
- 전세금 = 월세*12/이자율
- 전월세 전환율이라는 이자율을 사용해서 저기 이자율에 집어 넣어야함
- 지금 사용할 전월세 전환율=0.049, 보통 시중은행보다 약간 높다.

In [4]:
# 전세-월세 전환
df['deposit1']=df['m_rent']*12/0.049
df

,Unnamed: 0,monoth,deposit,m_rent,built_yr,dong_name,apt_name,size,gu_code,deposit1
0,0,202312,"1,000",80,2018,신설동,포레스트힐시티,14.750,11230,19591.836735
1,1,202312,"1,000",77,2017,신설동,신설동지웰홈스,19.700,11230,18857.142857
2,2,202312,"1,000",70,2017,신설동,신설동지웰홈스,13.750,11230,17142.857143
3,3,202312,"16,200",10,2018,신설동,포레스트힐시티,15.250,11230,2448.979592
4,4,202312,"3,000",56,2018,신설동,포레스트힐시티,15.210,11230,13714.285714
...,...,...,...,...,...,...,...,...,...,...
670,670,202312,"37,000",0,2002,이문동,중앙하이츠빌,59.400,11230,0.000000
671,671,202312,"5,000",100,2000,이문동,쌍용,59.990,11230,24489.795918
672,672,202312,"37,000",0,2002,이문동,중앙하이츠빌,59.400,11230,0.000000
673,673,202312,"54,000",0,2001,이문동,이문삼성래미안아파트,149.934,11230,0.000000


In [5]:
# 0번 : 1000/80은 전세 1억9천5백+천만원=2억5백 정도 하는것

# 전세금 다시 정의 = deposit1+보증금
# 근데 deposit을 문자로 이해하고 있음 .콤마가 있어서
df['deposit_new']=df['deposit']+df['deposit1']
df

TypeError: can only concatenate str (not "float") to str

In [6]:
# 먼저 deposit을 숫자로 변환하자. 이런 코드는 외워놓자
df['deposit']=df['deposit'].str.replace(",","").astype(int)
df['deposit_new']=df['deposit']+df['deposit1']
df

,Unnamed: 0,monoth,deposit,m_rent,built_yr,dong_name,apt_name,size,gu_code,deposit1,deposit_new
0,0,202312,1000,80,2018,신설동,포레스트힐시티,14.750,11230,19591.836735,20591.836735
1,1,202312,1000,77,2017,신설동,신설동지웰홈스,19.700,11230,18857.142857,19857.142857
2,2,202312,1000,70,2017,신설동,신설동지웰홈스,13.750,11230,17142.857143,18142.857143
3,3,202312,16200,10,2018,신설동,포레스트힐시티,15.250,11230,2448.979592,18648.979592
4,4,202312,3000,56,2018,신설동,포레스트힐시티,15.210,11230,13714.285714,16714.285714
...,...,...,...,...,...,...,...,...,...,...,...
670,670,202312,37000,0,2002,이문동,중앙하이츠빌,59.400,11230,0.000000,37000.000000
671,671,202312,5000,100,2000,이문동,쌍용,59.990,11230,24489.795918,29489.795918
672,672,202312,37000,0,2002,이문동,중앙하이츠빌,59.400,11230,0.000000,37000.000000
673,673,202312,54000,0,2001,이문동,이문삼성래미안아파트,149.934,11230,0.000000,54000.000000


In [9]:
# deposit_new를 바로 비교하면 안됨
# 평당 전세가로 바꿔서 비교해야함

# 알아서 해보세요

# 평당 전세가(보증금) 구하기
df['size_p']=df['size']/3.3 
df['deposit_p']=df['deposit_new']/df['size_p']
df

,Unnamed: 0,monoth,deposit,m_rent,built_yr,dong_name,apt_name,size,gu_code,deposit1,deposit_new,size_p,deposit_p
0,0,202312,1000,80,2018,신설동,포레스트힐시티,14.750,11230,19591.836735,20591.836735,4.469697,4606.987202
1,1,202312,1000,77,2017,신설동,신설동지웰홈스,19.700,11230,18857.142857,19857.142857,5.969697,3326.323423
2,2,202312,1000,70,2017,신설동,신설동지웰홈스,13.750,11230,17142.857143,18142.857143,4.166667,4354.285714
3,3,202312,16200,10,2018,신설동,포레스트힐시티,15.250,11230,2448.979592,18648.979592,4.621212,4035.516895
4,4,202312,3000,56,2018,신설동,포레스트힐시티,15.210,11230,13714.285714,16714.285714,4.609091,3626.373626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,670,202312,37000,0,2002,이문동,중앙하이츠빌,59.400,11230,0.000000,37000.000000,18.000000,2055.555556
671,671,202312,5000,100,2000,이문동,쌍용,59.990,11230,24489.795918,29489.795918,18.178788,1622.209144
672,672,202312,37000,0,2002,이문동,중앙하이츠빌,59.400,11230,0.000000,37000.000000,18.000000,2055.555556
673,673,202312,54000,0,2001,이문동,이문삼성래미안아파트,149.934,11230,0.000000,54000.000000,45.434545,1188.522950


In [10]:
# ln3 마지막 문제 해보기 !! 꼭 해보기!!

# LN4 : API와 주가데이터(1)

## 1 확률과 확률 법칙

다 필기, 강의 참고

In [12]:
# random drawing with replacement: 복원 추출
# random drawing without replacement: 비복원추출

# random package 먼저 설치

pip install randoms

SyntaxError: invalid syntax (3882665204.py, line 6)

In [14]:
import random
# 파이썬 내장 모듈임

In [15]:
box=[1,2,3,4,5] # 리스트 생성
box

[1, 2, 3, 4, 5]

In [17]:
sample_with_replacement=random.choices(box,k=2) # with replacement 복원 추출 명령어, k=2: 두개 무작위로 뽑겠다는거
sample_with_replacement

[2, 5]

In [19]:
# 교수님과 다른 번호 다름
# 같게 하려면 고정시키면됨 

random.seed(123)
sample_with_replacement=random.choices(box,k=2)
sample_with_replacement

[1, 1]

In [20]:
# without replacement: 비복원추출

random.seed(42)
sample_without_replacement=random.sample(box,k=2)
sample_without_replacement

[1, 5]

In [22]:
# 근데 현실에선 이 추출을 여러번 할 것
# 반복하려면 loop쓰면 됨

for i in range(10): # 10번 반복하라는 것 (0번, 1번 , ..., 9번까지 총 10번 하는 것)
    sample_without_replacement=random.sample(box,k=2)
    print(sample_without_replacement)

[5, 3]
[1, 2]
[4, 3]
[3, 2]
[2, 3]
[1, 5]
[4, 1]
[3, 5]
[5, 3]
[1, 4]


In [23]:
# 데이터를 반복해서 수집해! 도 응용하면 할 수 있다?!
# 어케해??
# 생각해볼것 꼭!!!!